In [1]:
import pandas as pd

data = pd.read_csv('data/non_annualized_GA_KY_lo_ml_md_mh_hi.csv')
data.head()

,Unnamed: 0,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),...,Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Harvested in Sown Year,First Date of Season,Percent Cover (%),Unnamed: 19,Class
0,136,Georgia,Athens,10/10/2007,Bara 503,8/20/2010,232,2.63,1045,38,...,238.752,27.202632,22.902632,33.138158,0.118289,0,0,72.352459,NaN,5
1,111,Georgia,Athens,10/10/2007,BaraWet 501,8/20/2010,232,2.61,1045,38,...,238.752,27.202632,22.902632,33.138158,0.118289,0,0,83.441803,NaN,5
2,1280,Kentucky,Lexington,3/31/2015,Caliber,5/6/2016,127,2.61,402,232,...,105.900,12.432653,6.540816,18.083673,0.273041,0,1,96.238342,NaN,5
3,116,Georgia,Athens,10/10/2007,CW 500,8/20/2010,232,2.58,1045,38,...,238.752,27.202632,22.902632,33.138158,0.118289,0,0,76.815574,NaN,5
4,121,Georgia,Athens,10/10/2007,Phoenix,8/20/2010,232,2.48,1045,38,...,238.752,27.202632,22.902632,33.138158,0.118289,0,0,74.027049,NaN,5


In [ ]:
#from sdv.tabular import CTGAN

#model = CTGAN()
#model.fit(data)

In [ ]:
#new_data = model.sample(1000)

In [ ]:
#new_data.to_csv('data/non_annualized_synth3_GA_KY_1K.csv')
#new_data.head()

In [2]:
# get aggregate data
#targetDataLoc = 'data/GA_only_063022.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = data #pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [3]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVC  # support vector machines classification
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsClassifier # k-nearest neightbors for classification
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # adaboost for classification
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [4]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Class"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = aggDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = aggDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [5]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["gini"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svcParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

logRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['gini'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [#(RandomForestClassifier(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsClassifier(), knnParamGrid, "knnTup.pkl"),
              (SVC(), svcParamGrid, "svrTup.pkl"),
             #(MLPRegressor(), nnParamGrid, "nnTup.pkl")],
             (LogisticRegression(), logRegParamGrid, "linRegTup.pkl")]#,
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             #(DecisionTreeClassifier(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 8 # 11

In [6]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  knnTup
Avg MAE:  602.83
Avg R squared:  0.84
Best MAE:  472.441
Best R squared:  0.885
Parameters of the best model:  {'model__leaf_size': 30, 'model__n_neighbors': 10, 'model__weights': 'distance'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
Accuracy:  0.6771653543307087
f1 score:  0.6696026093721946
mcc:  0.6060598326980132
 
test results on our test data: 
results:
0.24588880187940484
0.8693083275917841
model:  svrTup
Avg MAE:  709.203
Avg R squared:  0.781
Best MAE:  488.189
Best R squared:  0.882
Parameters of the best model:  {'model__C': 10.0, 'model__degree': 2, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moi